# generar imagen de referencia 
1. Se propone una imagen de referencia con un estado inicial que se va actualizando con  imagen a  imagen
2. 

In [ ]:
import numpy as np
class imgRef():
    def __init__(self,shape,altura=12000,ro=1000):
        self.shape=shape
        self.razon_cresimiento=10/100
        self.razon_decresimiento=0.1/100
        self.altura=altura
        self.img_ref=altura*np.ones((shape[0],shape[1])).reshape(-1)
        self.ro=ro
    def actualizar(self,img):
        diferencia=self.img_ref-img.copy()
        diferencia=diferencia.reshape(-1)
        for i in range(len(diferencia)):
            if np.absolute(diferencia[i])<self.ro:
                if diferencia[i]<0:
                    self.img_ref[i]+=diferencia[i]*self.razon_cresimiento
                else:
                    self.img_ref[i]-=diferencia[i]*self.razon_decresimiento
        return self.img_ref.reshape(self.shape[0],self.shape[1]).astype(np.uint16)


# comportamiento de la imagen referencia

In [ ]:
import glob
import cv2
from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline
import numpy as np
from matplotlib import cm
import matplotlib.pyplot as plt
import sys# las sisguientes dos lineas ayudan a ubicar al archivo calibracion.py y segementador.py
sys.path.append("/home/estufab4/Desktop/flujo de bagazo/codigo/calibracion/")
sys.path.append("/home/estufab4/Desktop/flujo de bagazo/codigo/segmentacion/")
from calibracion import graficar, plot
from segmentacion import AbrirSegmentador
########################################################


########################################################
path_depth="/home/estufab4/Desktop/flujo de bagazo/BASE_DATOS/DEPTH/"
path_rgb="/home/estufab4/Desktop/flujo de bagazo/BASE_DATOS/RGB/"
referencia=imgRef([480,640],altura=18000)
img_ref=referencia.img_ref.reshape(480,640)
plt.clf()
segmentador=AbrirSegmentador(pathConfig="{}traking/".format(path_rgb),
                                       nombre="config")

print(img_ref.shape)
for i in range(1,100):
    y,x=np.mgrid[0:img_ref.shape[0],0:img_ref.shape[1]]
    fig = plt.figure(figsize=(20,20))
    ax = Axes3D(fig)
    ax.plot_surface(y, x, img_ref, rstride=1, cstride=1, cmap=cm.coolwarm)
    ax.view_init(-30, 10)
    plt.show()
    directorio = glob.glob("{}{}_*.png".format(path_depth,i))
    print(directorio[0])
    depth=cv2.imread(directorio[0],cv2.IMREAD_UNCHANGED)[:,:,0]
    img_ref=referencia.actualizar(depth.reshape(-1))
   
    print("promedio: ",img_ref.mean(),"max: ",depth.max())
    graficar([depth],1,1)
   
    

In [ ]:

from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter
import numpy as np


fig = plt.figure()
ax = fig.gca(projection='3d')

# Plot the surface.
surf = ax.plot_surface(y, x, img_ref, cmap=cm.coolwarm,
                       linewidth=0, antialiased=False)

# Customize the z axis.
ax.set_zlim(-1.01, 1.01)
ax.zaxis.set_major_locator(LinearLocator(10))
ax.zaxis.set_major_formatter(FormatStrFormatter('%.02f'))

# Add a color bar which maps values to colors.
fig.colorbar(surf, shrink=0.5, aspect=5)

plt.show()
print(img_ref.min())

In [ ]:
import glob
import cv2
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import sys# las sisguientes dos lineas ayudan a ubicar al archivo calibracion.py y segementador.py
sys.path.append("/home/estufab4/Desktop/flujo de bagazo/codigo/calibracion/")
sys.path.append("/home/estufab4/Desktop/flujo de bagazo/codigo/segmentacion/")
from calibracion import graficar, plot
from segmentacion import AbrirSegmentador
########################################################


########################################################
path_depth="/home/estufab4/Desktop/flujo de bagazo/BASE_DATOS/DEPTH/"
path_rgb="/home/estufab4/Desktop/flujo de bagazo/BASE_DATOS/RGB/"
referencia=imgRef([480,640],altura=15000,ro=200)
img_ref=referencia.img_ref.reshape(480,640)
plt.clf()
segmentador=AbrirSegmentador(pathConfig="{}".format(path_rgb),
                                       nombre="config")

print(img_ref.shape)
for i in range(1,100):
    y,x=np.mgrid[0:img_ref.shape[0],0:img_ref.shape[1]]
    fig = plt.figure(figsize=(20,20))
    ax = Axes3D(fig)
    ax.plot_surface(y, x, img_ref, rstride=1, cstride=1, cmap=cm.coolwarm)
    ax.view_init(-30, 10)
    plt.savefig('img/{}.png'.format(i))    
    directorio = glob.glob("{}{}_*.png".format(path_depth,i))
    print(directorio[0])
    depth=cv2.imread(directorio[0],cv2.IMREAD_UNCHANGED)[:,:,0]
    directorio = glob.glob("{}{}_*.png".format(path_rgb,i))
    rgb=cv2.imread(directorio[0])
    
    mascara,_=segmentador.segmentar(rgb)
    depth=cv2.bitwise_and(depth,depth,mask=mascara)
    rgb=cv2.bitwise_and(rgb,rgb,mask=mascara)
    img_ref=referencia.actualizar(depth.reshape(-1))
    print("promedio: ",img_ref.mean(),"max: ",depth.max())
    #####################################################
    #flip vertical en visualización para correspondencia de lo que se visualiza
    depth=cv2.flip(depth, 0)
    rgb=cv2.flip(rgb, 0)
    mascara=cv2.flip(mascara, 0)
    graficar([depth,rgb,mascara],1,3)
    plt.savefig('img/{}imgs.png'.format(i))
   

In [ ]:
import glob
import cv2
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import sys# las sisguientes dos lineas ayudan a ubicar al archivo calibracion.py y segementador.py
sys.path.append("/home/estufab4/Desktop/flujo de bagazo/codigo/calibracion/")
sys.path.append("/home/estufab4/Desktop/flujo de bagazo/codigo/segmentacion/")
from calibracion import graficar, plot
from segmentacion import AbrirSegmentador
########################################################


########################################################
path_depth="/home/estufab4/Desktop/flujo de bagazo/BASE_DATOS/DEPTH/"
path_rgb="/home/estufab4/Desktop/flujo de bagazo/BASE_DATOS/RGB/"
referencia=imgRef([480,640],altura=15000,ro=50)
img_ref=referencia.img_ref.reshape(480,640)
plt.clf()
segmentador=AbrirSegmentador(pathConfig="{}".format(path_rgb),
                                       nombre="config")

print(img_ref.shape)
for i in range(1,2000):

#     plt.savefig('img/{}.png'.format(i))    
    directorio = glob.glob("{}{}_*.png".format(path_depth,i))
#     print(directorio[0])
    depth=cv2.imread(directorio[0],cv2.IMREAD_UNCHANGED)[:,:,0]
    directorio = glob.glob("{}{}_*.png".format(path_rgb,i))
    rgb=cv2.imread(directorio[0])
    
    mascara,_=segmentador.segmentar(rgb)
    depth=cv2.bitwise_and(depth,depth,mask=mascara)
    rgb=cv2.bitwise_and(rgb,rgb,mask=mascara)
    img_ref=referencia.actualizar(depth.reshape(-1))
#     print("promedio: ",img_ref.mean(),"max: ",depth.max())
    #####################################################
    #flip vertical en visualización para correspondencia de lo que se visualiza
    depth=cv2.flip(depth, 0)
    rgb=cv2.flip(rgb, 0)
    mascara=cv2.flip(mascara, 0)
graficar([depth,rgb,mascara,img_ref],2,3)
y,x=np.mgrid[0:img_ref.shape[0],0:img_ref.shape[1]]
fig = plt.figure(figsize=(20,20))
ax = Axes3D(fig)
ax.plot_surface(y, x, img_ref, rstride=1, cstride=1, cmap=cm.coolwarm)
ax.view_init(-30, 10)
plt.savefig('img/{}_img_ref.png'.format(referencia.ro))

In [ ]:
# generación de imagen de referencia con ayuda de la segmentación adaptativa

In [ ]:
import glob
from skimage import color
from sklearn.preprocessing import normalize, scale
from sklearn.cluster import KMeans
from skimage import morphology
from tabulate import tabulate
import sys# las sisguientes dos lineas ayudan a ubicar al archivo calibracion.py
sys.path.append("/home/estufab4/Desktop/flujo de bagazo/codigo/calibracion/")
from calibracion import *
import random
from ttictoc import tic, toc 
import cv2
from scipy import stats
class Segmentador:
    def __init__(self,n_clusters):
        self.centroides=0
        self.kmeans=KMeans(n_clusters) # obj para reducir colores
#         print(self.kmeans.cluster_centers_)
        self.n_clusters=n_clusters
        self.kernel = np.ones((3,3),np.uint8)
    def entrenar(self, img):
        gray=cv2.cvtColor(img.copy(), cv2.COLOR_BGR2HSV)
        img=cv2.medianBlur(img,17)
        hsv = cv2.cvtColor(img.copy(), cv2.COLOR_BGR2HSV)
        
        canal_h=hsv[:,:,0]
#         print(canal_h.max())
        canal_h=(255/canal_h.max())*canal_h-0.2*hsv[:,:,2]
        canal_h=cv2.morphologyEx(canal_h, cv2.MORPH_CLOSE, self.kernel)
        canal_h=cv2.medianBlur(canal_h.astype(np.uint8),3)
        canal_h=(canal_h-canal_h.min())/(canal_h.max()-canal_h.min())
        shape=canal_h.shape
#         tic()
        if hasattr(self.kmeans, 'cluster_centers_'):
            self.kmeans=KMeans(n_clusters=self.n_clusters, init=self.kmeans.cluster_centers_, n_init=1,max_iter=300, tol=0.001, random_state=True, copy_x=True, n_jobs=-1, algorithm='auto').fit(canal_h.reshape(-1,1)) 
        else:
            self.kmeans=KMeans(n_clusters=self.n_clusters, init='k-means++', n_init=10, max_iter=300, tol=0.001, random_state=True, copy_x=True, n_jobs=-1, algorithm='auto').fit(canal_h.reshape(-1,1)) 
        img_reducida=np.choose( self.kmeans.labels_, self.kmeans.cluster_centers_)
        img_reducida.shape=shape
        moda=stats.mode(img_reducida,axis=None)
        mascara=img_reducida.copy()
        mascara[mascara==moda[0]]=1
        mascara[mascara!=1]=0
        mascara=mascara.astype(np.bool_)## remove_small solo funciona con variables binarias
        
        mascara=morphology.remove_small_objects(mascara,200,connectivity=1)
        mascara=mascara.astype(np.uint8)
        mascara=cv2.morphologyEx(mascara, cv2.MORPH_CLOSE, self.kernel)
        return mascara,img_reducida


In [ ]:
import glob
import cv2
import numpy as np
path="/home/estufab4/Desktop/flujo de bagazo/codigo/img_dataset/fondo_MY3/"
newpath="/home/estufab4/Desktop/flujo de bagazo/codigo/img_dataset/MY/"
dirrgb= glob.glob("{}RGB*.png".format(path))
dirdepth=[]
for name in dirrgb:
    index=name.find("RGB")
    new=newpath+"depth"+name[index+3:]
    img=cv2.imread(new,cv2.IMREAD_UNCHANGED)
    dirdepth.append(path+"depth"+name[index+3:])
    cv2.imwrite(path+"depth"+name[index+3:],img)

In [ ]:
depth

In [ ]:
import glob
import cv2
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import sys# las sisguientes dos lineas ayudan a ubicar al archivo calibracion.py y segementador.py
sys.path.append("/home/estufab4/Desktop/flujo de bagazo/codigo/calibracion/")
sys.path.append("/home/estufab4/Desktop/flujo de bagazo/codigo/segmentacion/")
from calibracion import graficar, plot
from segmentacion import AbrirSegmentador

########################################################
def recuperar(img):
        mask=img.copy()
        mask[mask<=300]=1
        mask[mask!=1]=0
        mask=mask.astype(np.uint8)
        dst = cv2.inpaint(img,mask,1,cv2.INPAINT_TELEA)
#         graficar([dst],1,1)
        return dst

########################################################
path="/home/estufab4/Desktop/flujo de bagazo/codigo/img_dataset/fondo_MY3/"
referencia=imgRef([480,640],altura=18000,ro=100)
img_ref=referencia.img_ref.reshape(480,640)
plt.clf()
segmentador=Segmentador(3)

print(img_ref.shape)
for imgRGB,imgZ in zip(dirrgb,dirdepth):

    depth=cv2.imread(imgZ,cv2.IMREAD_UNCHANGED)[:,:,0]
    depth=recuperar(depth)
    print(depth.max(),depth.min())
    rgb=cv2.imread(imgRGB)
    
    mascara,_=segmentador.entrenar(rgb)
    depth=cv2.bitwise_and(depth,depth,mask=mascara)
    rgb=cv2.bitwise_and(rgb,rgb,mask=mascara)
    img_ref=referencia.actualizar(depth.reshape(-1))
#     print("img:",i)
    print("promedio: ",img_ref.mean(),"max: ",depth.max())
    #####################################################
    #flip vertical en visualización para correspondencia de lo que se visualiza
print(img_ref.dtype,depth.dtype)
path_save="/home/estufab4/Desktop/flujo de bagazo/codigo/img_dataset/fondo_MY3/"
cv2.imwrite("{}ro_{}_img_ref.png".format(path_save,referencia.ro),img_ref)
depth=cv2.flip(depth, 0)
rgb=cv2.flip(rgb, 0)
mascara=cv2.flip(mascara, 0)
graficar([depth,rgb,mascara,img_ref],2,3)
y,x=np.mgrid[0:img_ref.shape[0],0:img_ref.shape[1]]
fig = plt.figure(figsize=(20,20))
ax = Axes3D(fig)
ax.plot_surface(y, x, img_ref, rstride=1, cstride=1, cmap=cm.coolwarm)
ax.view_init(-30, 10)
path_save="/home/estufab4/Desktop/flujo de bagazo/codigo/img_dataset/fondo_MY3/"
print(img_ref.max())

plt.savefig('{}ro_{}_plt_ref.png'.format(path_save,referencia.ro))

In [ ]:
print("{}{}_img_ref.png".format(path_save,referencia.ro))
img_ref

# medición de bagazo


In [ ]:
import numpy as np
from scipy.signal import savgol_filter
import cv2
import glob
import matplotlib.pyplot as plt
from scipy import stats
from ttictoc import tic, toc 
class Traking:
    def __init__(self,area_surco=15,rango_salto_permitido=50,distancia_min_entre_surcos=100):
        #variable/ variables de instancia
        self.area_surco=area_surco# surco o region de interes ROI
        self.rango_salto_permitido=rango_salto_permitido# No habrá detección cuando el valor minimo anterior y el valor minimo actual del histograma haya una diferencia dada
        self.distancia_min_entre_surcos=distancia_min_entre_surcos# este parametro impide detectar el segundo surco cerca del primero o cerca del final de la imagen
        self.deteccion=0 # se pronone esta variable para evitar que el valor del histograma mas bajo entre a la RIO salga y vulva a entrar
        self.buffer_minimo=0# util para descartar detecciones incorrectas. mediante una resta se compara el actual y el aterior buscando que no hayan cambios bruscos
        self.cnt=0
        self.ndeteccion=10# nuero de imagenes minimo en el que no se espera detección luego de haber ocurrido una.
    def extraccion(self,frame):# ret indicara si ha ocurrido una deteccion. En caso de no encontrar el surco alrededor de la fila cero se retorna una imagen en negro
        #variables 
        img= cv2.cvtColor(frame.copy(), cv2.COLOR_BGR2GRAY)
        # analisis
        histo_horiz=np.sum(img,axis=1)/img.shape[1]
        primersurco=histo_horiz[:self.distancia_min_entre_surcos]
        index_minimo=list(primersurco).index(np.amin(primersurco))# fila con intencidad de pixeles minimo
#         print(index_minimo)
        # el minimo actual debe entrar a la regio de interes, entre el minimo anterior y el actual no debe haber una diferencia de mas del valor permitido, el minimo anterior no debe estar dentro de la region de interes
        if (index_minimo<=self.area_surco)and(np.absolute(self.buffer_minimo-index_minimo)<self.rango_salto_permitido)and(self.buffer_minimo>self.area_surco and self.deteccion<=0):
            self.deteccion=self.ndeteccion+1
            ret=True
            histo_horiz=self.filtroHisto(histo_horiz)
            # una vez encontrado el surco alrededor de la fila cero, se procede a encontrar el segundo surco que limite el lote del bagazo
#             img[:self.distancia_min_entre_surcos+1,:]=255# este punto marcara el pico de otro lote de bagazo, el nuevo surco debera estar entre el primer surco y el maximo del segundo lote
            segundosurco=histo_horiz[self.distancia_min_entre_surcos:-int(self.distancia_min_entre_surcos+0.7*self.distancia_min_entre_surcos)]
            x=np.arange(len(segundosurco))+self.distancia_min_entre_surcos
            cambio_gradiente=np.gradient(np.gradient(segundosurco))
#             plt.plot(x,segundosurco)
#             plt.show()
#             plt.plot(x,cambio_gradiente)
#             plt.show()
            recorte=np.where(cambio_gradiente==np.amax(cambio_gradiente))
            #recorte del area de interes
            img=frame
#             img[self.distancia_min_entre_surcos+recorte[0][0]+1:,:,:]=0
            img=img[:self.distancia_min_entre_surcos+recorte[0][0]+1,:,:]

        else: # en caso de que la fila de minima dencidad no este alrededor de cero 
            img=np.zeros(frame.shape)
            if self.deteccion>0:
                self.deteccion-=1
            ret=False
        # actualizar fila de minima intencidad
        self.buffer_minimo=index_minimo
        return ret,img
    def filtroHisto(self,data):
#         tic()
        data=savgol_filter(data, 11, 3, mode='nearest')
#         print(toc())
        return data

In [ ]:
import numpy as np
import cv2
import sys# las sisguientes dos lineas ayudan a ubicar al archivo calibracion.py
sys.path.append("/home/estufab4/Desktop/flujoBagazo/codigo/calibracion/")
from calibracion import graficar
from medicion import *
import matplotlib.pyplot as plt
import numpy as np
class Volumen(imagen):###################3 volumen en cm3
    def __init__(self,path_configuracion,nombre_configuracion):
      
        imagen.__init__(self,path_configuracion,nombre_configuracion,pendientes2=True)
        self.beta=np.array([ 0.12616642,  0.06137007, -0.00176466])# coeficientes de la calibración de area por optimización
        self.maxZ=25000
        self.minZ=3000
    def modeloArea(self,imgz):
        area=self.beta[0]*imgz**2+self.beta[1]*imgz+self.beta[2]
        return area
 
    def preprocesar_imgs(self,img_rgb,img_depth,f_segmentador,alto=0):
        if alto:
            img_depth[:alto,:]=self.recuperar(img_depth[:alto,:])
        else:
#         buffer=img_depth.copy()
        #quitar distorsion z
            img_depth=self.recuperar(img_depth)
        img_depth=self.distorsion_z2(img_depth,filtrar=False)
#         graficar([img_depth,buffer],1,2)
        #quitar distorsion uv
        img_rgb,img_depth=self.distorsion(img_rgb,img_depth)
        # cambiar de dominio rgb a depth
        img2_depth=self.rgb2_depth(img_rgb)
        # encontrar mascara
        mascara=f_segmentador(img2_depth)
        print(mascara.shape,img_depth.shape)
        
        return img_depth,mascara
    def calcular_volumen(self,img,mascara,img_ref):
      
        img_z=img.reshape(-1)
        mascara=mascara.reshape(-1)
        img_ref=img_ref.reshape(-1)
        # necesario para compatibilidad de dimensiones
        # extraer los pixeles del objetivo
        index=np.where(mascara==1)
        img_z=img_z[index]
        img_ref=img_ref[index]
        print(img_ref.mean(),"ref_img")
        
#         graficar([img_z,img_ref],1,2,["img x mascara","ref"])
        znorm=self.normalizarz(img_z)
        img_area=self.modeloArea(znorm)
        print(img_area.min(),img_area.max(),'area')
#         print(img_area.su,m(),"AREA TOTAL")
        
        img_altura=img_ref-img_z
        print(img_altura.min(),img_altura.max(),'altura')
        img_altura=img_altura/100 # pasar a cm
#         print(img_altura.mean(),"ALTURA MEDIA")
        img_volumen=img_area*img_altura
        volumen=np.sum(img_volumen)
#         print(volumen, "Volumen estimado")
        return volumen
    def calcular_error(self,valor_real,valor_medido):
        e=100*(valor_real-valor_medido)/valor_real
        return e
    def mejorar_referencia(self,img_ref):

#         img_ref[img_ref==0]=img_ref.mean()
        img_ref=self.distorsion_z(img_ref,1)

        return img_ref
    def abrir_img_ref(self,path,nombre="",distorsion=True):
        ret=False
        if nombre=="":
            names = glob.glob("{}depth_*.png".format(path))
            ret=True if len(names)>0 else False
            imgRef=0
            for name in names:
                img=cv2.imread(name,cv2.IMREAD_UNCHANGED)[:,:,0]
                img=self.recuperar(img)
                img=self.distorsion_z2(img,filtrar=False)
                imgRef+=img
            imgRef/=len(names)
        else:
            print("{}{}.png".format(path,nombre))
            imgRef=cv2.imread("{}{}.png".format(path,nombre),cv2.IMREAD_UNCHANGED)
            print(imgRef)
           
            if  hasattr(imgRef, 'shape'):
                ret=True
                if len(imgRef.shape)==3:
                    imgRef=imgRef[:,:,0]
                imgRef=self.recuperar(imgRef)
            if distorsion:
                imgRef=self.distorsion_z2(imgRef,filtrar=False)
        return ret,imgRef
    def normalizarz(self, z):
        z=(z-self.minZ)/(self.maxZ-self.minZ)
        return z

In [ ]:
# import  matplotlib.pyplot as plt
# import sys# las sisguientes dos lineas ayudan a ubicar al archivo calibracion.py
# sys.path.append("/home/estufab4/Desktop/flujo de bagazo/codigo/calibracion/")
# from calibracion import *
# import numpy as np
# import cv2
# from ttictoc import tic, toc 

# def segmentar(img):

#     n=7
#     kernel = np.ones((n,n),np.uint8)
#     img=cv2.medianBlur(img,7)

#     ############################################################################
#     gray=cv2.cvtColor(img.copy(), cv2.COLOR_BGR2GRAY)
#     thr, _= cv2.threshold(gray,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)

#     mascara=np.zeros_like(gray)
#     mascara[gray>thr]=1
#     # lo anterior deberia generar una cresta con dos valles como caracteristicas resaltantes en el histograma del canal h de HSV.
#     # otsu deberia indicar donde ocurren los valles, sin embargo dependera de los datos  que valle aparece primero
#     canalH=cv2.cvtColor(img.copy(), cv2.COLOR_BGR2HSV)[:,:,0]
#     mediaH=canalH[mascara==1].mean()
# #     plt.hist(canalH[canalH>mediaH].reshape(-1))
# #     plt.hist(canalH[canalH<mediaH].reshape(-1))
    
    
#     thrDer, _= cv2.threshold(canalH[canalH>=mediaH],0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
#     thrIz, _= cv2.threshold(canalH[canalH<mediaH],0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
# #     print(thrDer,thrIz)
#     mascara[canalH>thrDer]=0
#     mascara[canalH<thrIz]=0

#     n=15
#     kernel = np.ones((n,n),np.uint8)
#     mascara=cv2.morphologyEx(mascara, cv2.MORPH_CLOSE, kernel)

#     return mascara 


In [ ]:
import  matplotlib.pyplot as plt
import sys# las sisguientes dos lineas ayudan a ubicar al archivo calibracion.py
sys.path.append("/home/estufab4/Desktop/flujoBagazo/codigo/calibracion/")
from calibracion import *
import numpy as np
import cv2
from ttictoc import tic, toc 

def segmentar(img):
    tic()
    n=7
    kernel = np.ones((n,n),np.uint8)
    img=cv2.medianBlur(img,7)
    img[:,:90,:]=0
    img[:,590:,:]=0
#     img=cv2.morphologyEx(img, cv2.MORPH_CLOSE, kernel)
#     img=cv2.morphologyEx(img, cv2.MORPH_GRADIENT, kernel)
    ############################################################################
    img=cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    gray=cv2.cvtColor(img.copy(), cv2.COLOR_BGR2GRAY)
    thr, _= cv2.threshold(gray,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)

    mascara=np.zeros_like(gray)
    mascara[gray>thr]=1
    
    # lo anterior deberia generar una cresta con dos valles como caracteristicas resaltantes en el histograma del canal h de HSV.
    # otsu deberia indicar donde ocurren los valles, sin embargo dependera de los datos  que valle aparece primero
    canalH=cv2.cvtColor(img.copy(), cv2.COLOR_BGR2HSV)[:,:,0]
    mediaH=canalH[mascara==1].mean()
#     plt.hist(canalH[canalH>mediaH].reshape(-1))
#     plt.hist(canalH[canalH<mediaH].reshape(-1))
    
    
    thrDer, _= cv2.threshold(canalH[canalH>=mediaH],0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    thrIz, _= cv2.threshold(canalH[canalH<mediaH],0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
#     print(thrDer,thrIz)
    canalH[mascara==0]=0
    mascara[canalH>thrDer]=0
    mascara[canalH<thrIz]=0
#     mascara=mascara.astype(np.bool_)## remove_small solo funciona con variables binarias
        
#     mascara=morphology.remove_small_objects(mascara,200,connectivity=1)
#     mascara=mascara.astype(np.uint8)
    n=15
    kernel = np.ones((n,n),np.uint8)
    mascara=cv2.morphologyEx(mascara, cv2.MORPH_CLOSE, kernel)
    
    
    
        
    
#     imgInv=img.copy()
#     imgInv[mascara==1]=0
#     img[mascara==0]=0
#     graficar([img,imgInv,gray,canalH],1,4)
    return mascara 


In [ ]:
import numpy as np
from datetime import datetime
import pandas as pd
import glob
import matplotlib.pyplot as plt

def getDataFrame(path:str,prefix:str):
    fname=glob.glob("{}{}*.png".format(path,prefix))
    lenPrefix=len(prefix)+1
    print(len(fname))
    dates=[]
    index=[]
    for name in fname:
        #extract index
        iini=name.find(prefix)
        iend=name[iini+lenPrefix:].find("_")
        i=int(name[iini+lenPrefix:iini+lenPrefix+iend])
        index.append(i)
        #extract datetime
        date=name[iini+lenPrefix+iend+1:-4]
        dates.append(datetime.strptime(date, '%m_%d_%Y_%H_%M_%S'))
    df=pd.DataFrame(list(zip(index,dates,fname)),columns=["prefix","dates","fname"])
#     plt.plot(df["dates"],df["prefix"])
    df=df.sort_values(["dates","prefix"])
    plt.figure(figsize=(15,10))
    plt.plot(df["dates"],df["prefix"])
    return df
def filtrarHora(df,mesdia, ini,fin):
    dateini=datetime.strptime("2021-{} {}".format(mesdia,ini),'%Y-%m-%d %H:%M')
    datefin=datetime.strptime("2021-{} {}".format(mesdia,fin),'%Y-%m-%d %H:%M')

    df=df[(df["dates"]>dateini) & (df["dates"]<datefin) ]
  
    return list(df["fname"])


In [ ]:
path="/home/estufab4/Desktop/flujoBagazo/dataset/img_dataset_bagazo/risaralda/septiembre/29/verde/"
nombresRGB=getDataFrame(path,"RGB")
nombresRGB=filtrarHora(df=nombresRGB,mesdia="09-29", ini="13:00",fin="14:00")
nombresDepth=getDataFrame(path,"depth")
nombresDepth=filtrarHora(df=nombresDepth,mesdia="09-29", ini="13:00",fin="14:00")
print(len(nombresDepth),len(nombresRGB))

In [ ]:
nombresRGB[0]

In [ ]:
import glob
import os.path as path
import sys# las sisguientes dos lineas ayudan a ubicar al archivo calibracion.py
sys.path.append("/home/estufab4/Desktop/flujoBagazo/codigo/calibracion/")
from calibracion import *
import matplotlib.pyplot as plt
# path_config="/home/estufab4/Desktop/flujo de bagazo/codigo/calibracion/configuracion_union_120/"
# config= "configuracion_d5_90"
path_config="/home/estufab4/Desktop/flujoBagazo/codigo/calibracion/configuraciónPrueba/"
# "/home/estufab4/Desktop/flujo de bagazo/codigo/calibracion/configuracion_union_150/"
config="distorsionz"
# path_img="/home/estufab4/Desktop/flujo de bagazo/codigo/img_dataset/MY3_1/"
# path_img="/home/estufab4/Desktop/flujo de bagazo/codigo/img_dataset/abrir2021/"
path_img="/home/estufab4/Desktop/flujoBagazo/dataset/img_dataset_bagazo/risaralda/septiembre/29/verde/"
path_ref="/home/estufab4/Desktop/flujoBagazo/dataset/img_dataset_bagazo/risaralda/septiembre/29/fondo2/"
nombre_img_ref="fondo"
densidad=1
medicion=Volumen(path_config,config)
ret,img_ref=medicion.abrir_img_ref(path_ref,nombre_img_ref,False)

print("ret",ret)
plt.imshow(img_ref)
plt.show()
print(img_ref.min(),img_ref.max())

traking=Traking(area_surco=10,rango_salto_permitido=50,distancia_min_entre_surcos=100)
masas=[]
print("procesando..")
cnt=0
bufferHora=""
tendencia=[]
totalMin=[]
for i in range(len(nombresDepth)) :

#     frgb=glob.glob("{}RGB_{}_*.png".format(path_img,i))
#     fdepth=glob.glob("{}depth_{}_*.png".format(path_img,i))
    #comprobar existencia
#     ret=path.isfile(fdepth[0])and path.isfile(frgb[0])
    ret=path.isfile(nombresRGB[i])and path.isfile(nombresDepth[i])
#     print(ret)
    if ret:
    
#         img_rgb=cv2.imread(frgbnombresDepth[0])
        img_rgb=cv2.imread(nombresRGB[i])
    
        img_rgb=cv2.cvtColor(img_rgb, cv2.COLOR_BGR2RGB)
        ret,img_proce=traking.extraccion(img_rgb)
#         print("detec:",ret)
        if ret:
            shape=img_proce.shape
            
            img_depth=cv2.imread(nombresDepth[i],cv2.IMREAD_UNCHANGED)
            if len(img_depth.shape)==3:
                img_depth=img_depth[:,:,0]
            
            tic()
            
            img_depth,mascara=medicion.preprocesar_imgs(img_proce,img_depth,segmentar,alto=shape[0])
#             graficar([img_depth],1,1,["Imagen de profundidad"])
            img_depth=img_depth[:shape[0],:]
#             mascara=segmentar(img_proce)
            img_depth=img_depth# conversion de 100um a  a metros
            
            volumen=medicion.calcular_volumen(img_depth,mascara,img_ref.copy())# cm3
            volumen=volumen/1000000
            print(volumen,'vol')
#             cv2.imshow("a",cv2.imread(nombresDepth[i])[:,:,0])
#             cv2.waitKey()
#             graficar([cv2.imread(nombresDepth[i])[:,:,0]],1,1)
            masa=volumen*densidad
            t=toc()
            print("tiempo",t)
            
            # crear tendencia por minuto,
            if bufferHora!=findMin(nombresRGB[i]):
                bufferHora=findMin(nombresRGB[i])
                totalMin=np.array(totalMin)
                tendencia.append(totalMin.sum())
                totalMin=[]
            totalMin.append(masa)
            masas.append(masa)
#             print(len(masas))
            if masa<9/160:
#                 print("segmentador:",segmentar.kmeans.cluster_centers_)
                print("% proceso: ",100*i/len(nombresDepth))
                print(nombresRGB[i])
                print(nombresDepth[i])
                img=img_proce.copy()
                img_proce[(1-mascara)==0]=0
                graficar([img,img_proce,mascara*img_depth,mascara],1,4,["RGB","RGBXinvMas","DEPTH x mascara","mascara"])
                print("volumen:",volumen)
                print("masa [kg]:",masa)
                print("recorte:",i)
#                 graficar([hsv[:,:,0],hsv[:,:,1],hsv[:,:,2]],1,3)
totalMin=np.array(totalMin)
tendencia.append(totalMin.sum())
tendencia=np.array(tendencia)                    
           
        
        
    

In [ ]:
# primeras mil imagenes
from scipy import stats
import  matplotlib.pyplot as plt
masas=np.array(masas)
print("número de lotes :",len(masas))
print("peso medio:", masas.mean())#Resultado/26.34046156332953 kg
print("peso máximo:", masas.max())#32.8
print("peso mínimo: ",masas.min())#19.55
moda=stats.mode((masas.astype(int)))
print("moda:{} Kg con {} casos de {}".format(moda[0],moda[1],len(masas)))# R/27kg con 12 casos de 68
plt.plot(masas)
plt.title("Peso x Lote")
plt.grid()
plt.xlabel("muestra (lote)", size = 16,)
plt.xlabel("Masa (Kg)", size = 16,)

In [ ]:
a=np.ones((shape[0],shape[1]))
print(a)
print(shape[0])

In [ ]:
# 1000-2000
from scipy import stats
import  matplotlib.pyplot as plt
masas=np.array(masas)
print("numero de lotes :",len(masas))
print("peso medio:", masas.mean())#Resultado/26.34046156332953 kg
print("peso maximo:", masas.max())#32.8
print("peso minimo: ",masas.min())#19.55
moda=stats.mode((masas.astype(int)))
print("moda:{} Kg con {} casos de {}".format(moda[0],moda[1],len(masas)))# R/27kg con 12 casos de 68
print("Total [kg]:",masas.sum())
plt.plot(masas)
plt.title("Peso x Lote")

In [ ]:
# 0-5000
# 1000-2000
from scipy import stats
import  matplotlib.pyplot as plt
masas=np.array(masas)
print("número de lotes :",len(masas))
print("peso medio:", masas.mean())#Resultado/26.34046156332953 kg
print("peso máximo:", masas.max())#32.8
print("peso mínimo: ",masas.min())#19.55
moda=stats.mode((masas.astype(int)))
print("moda:{} Kg con {} casos de {}".format(moda[0],moda[1],len(masas)))# R/27kg con 12 casos de 68
print("Total [kg]:",masas.sum())
# print("promedio por minuto kg:",masas.sum()/56)
# real=(389.03+90-360.20)*1000
# real=3192.797222222222# 11:55-12:00
real=2310.986111111111# 11:50-11:55
real=9189.155555555557# 11:40-12:00
real=8275.596794871823# 12:00-12:12
real=10137.597099197132#12:00_12:15
print("real 15 min [Kg]:",real)


error=100*(real-masas.sum())/real
print("error %",error)
plt.plot(masas)
plt.title("Peso x Lote")
plt.grid()
plt.xlabel("muestra (lote)", size = 16,)
plt.ylabel("Masa (Kg)", size = 16,)

In [ ]:
# 0-5000
# 1000-2000
from scipy import stats
import  matplotlib.pyplot as plt
masas=np.array(masas)
print("número de lotes :",len(masas))
print("peso medio:", masas.mean())#Resultado/26.34046156332953 kg
print("peso máximo:", masas.max())#32.8
print("peso mínimo: ",masas.min())#19.55
moda=stats.mode((masas.astype(int)))
print("moda:{} Kg con {} casos de {}".format(moda[0],moda[1],len(masas)))# R/27kg con 12 casos de 68
print("Total [kg]:",masas.sum())
# print("promedio por minuto kg:",masas.sum()/56)
# real=(389.03+90-360.20)*1000
# real=3192.797222222222# 11:55-12:00
real=2310.986111111111# 11:50-11:55
real=9189.155555555557# 11:40-12:00
real=8275.596794871823# 12:00-12:12
real=10137.597099197132#12:00_12:15
print("real 5 min [Kg]:",real)


error=100*(real-masas.sum())/real
print("error %",error)
plt.plot(masas)
plt.title("Peso x Lote")
plt.grid()
plt.xlabel("muestra (lote)", size = 16,)
plt.ylabel("Masa (Kg)", size = 16,)

In [ ]:
plt.plot(tendencia)

In [ ]:
import glob
import numpy as np
import cv2
path="/home/estufab4/Desktop/flujo de bagazo/codigo/img_dataset/MY3/"
path_save=["/home/estufab4/Desktop/flujo de bagazo/codigo/img_dataset/MY3_1/","/home/estufab4/Desktop/flujo de bagazo/codigo/img_dataset/MY3_2/"]
n=1
mult=100
for i in range(17500,224742):
    frgb=glob.glob("{}depth_{}_*.png".format(path,i))
    ordenar=[]
    for n in range(len(frgb)):
        index=frgb[n].find(format(i)+"_")# encontrar el indice donde enpieza la fecha
        identificador=int(frgb[n][index:-4])# quitar la parte que indica el formato, y converitr a un numero
        ordenar.append((n,identificador))# agrupar el indice y la fecha en una tupla
    ordenar=np.array(ordenar,dtype=[('index', int),('fecha', int)])# crear numpy con los formatos index y fecha
  # ordenar por el atributo fecha
    ordenar=np.sort(ordenar,order='fecha')
    index=range(len(ordenar))
    for ind,names in zip(index,ordenar):
        name=frgb[names[0]]
        img=cv2.imread(name,cv2.IMREAD_UNCHANGED)
        cambiar=name.find("MY3")
        nueva=path_save[ind]+name[cambiar+4:]
        cv2.imwrite(nueva,img)
    if i%100==0:
        print(i)
        print(name)

        

    

In [ ]:
print(66800)#rgb
print(17500)#depth

In [ ]:
import os.path as path
import cv2
path_img="/home/estufab4/Desktop/flujo de bagazo/codigo/solo_python/img_prueba/depth_20_10_05_2020_08_55_43.png"
path_img="/home/estufab4/Desktop/flujo de bagazo/codigo/img_dataset/MY/depth_3_09_18_2020_15_39_21.png"
path_img="/home/estufab4/Desktop/flujo de bagazo/codigo/img_dataset/fondo_MY3/ro_100_img_ref.png"
print(path.isfile(path_img))
img=cv2.imread(path_img,cv2.IMREAD_UNCHANGED)#[:,:,0]
print(img)
print(img.dtype)


# Generar imagen de referencia con img

In [ ]:
from sklearn import linear_model
import seaborn as sb
from medicion import *
def heatMap(img, vmin=0):
    if vmin:
        heat_map = sb.heatmap(img,yticklabels=False,xticklabels=False, vmin=vmin)
    else:
        heat_map = sb.heatmap(img,yticklabels=False,xticklabels=False)
    plt.show()
class GetRef(imagen):###################
    def __init__(self,path_configuracion,nombre_configuracion):
        imagen.__init__(self,path_configuracion,nombre_configuracion,pendientes2=True)
        self.shape=0
    def getPuntos(self,img,mask):
        if not(self.shape):
            self.shape=img.shape
        
        z=img[mask==255]

        #generar cordenadas uv. 
        v,u=np.mgrid[0:self.shape[0],0:self.shape[1]]

        umask=u[mask==255]
        vmask=v[mask==255]
        umask=umask[z!=0]
        vmask=vmask[z!=0]
        z=z[z!=0].reshape(-1,1)
        #Colocar datos en formato matriz columna (N,2)
        uvmask=np.array((umask.reshape(-1),vmask.reshape(-1))).transpose()

        return z,uvmask
    def calcularPlano(self,z,uv,shape):


        #generar cordenadas uv. 
        v,u=np.mgrid[0:self.shape[0],0:self.shape[1]]


        #Colocar datos en formato matriz columna (N,2)

        uvModelo=np.array((u.reshape(-1),v.reshape(-1))).transpose()



        modelo = linear_model.LinearRegression()
        #entrenar modelo

        modelo.fit(uv,z)


        #calcular z a partir del modelo
        z_modelo=modelo.predict(uvModelo)
        z_modelo=z_modelo.reshape(self.shape[0],self.shape[1])

        #plot(list_x=[range(len(z_modelo)),range(len(z_modelo))],list_y=[z,z_modelo],list_titulos=["valores datos por imagen"],ejes=[["index","valor camara"]],tipo=2)

        return z_modelo
    def ref(self,pathImg):
        names=glob.glob("{}RGB_*.png".format(pathImg))
        names=sorted(names)
        zModelo=np.array([]).reshape(-1,1)
        uvModelo=np.array([]).reshape(-1,2)
        for n in names:
            index=n.find("RGB")

            buffer=n[index+4:]
            index=buffer.find("_")
            num=buffer[:index]
            nameDepth=glob.glob("{}depth_{}_*.png".format(pathImg,num))[0]
            if nameDepth:
                print(n,nameDepth)
                imgRGB=cv2.imread(n)
                imgDepth=cv2.imread(nameDepth,cv2.IMREAD_UNCHANGED)
                if len(imgDepth.shape)==3:
                    imgDepth=imgDepth[:,:,0]
#                 print(imgDepth)
                
                imgDepth=self.recuperar(imgDepth)

                imgDepth=self.distorsion_z2(imgDepth,filtrar=False)

                #quitar distorsion uv
                imgRGB,imgDepth=self.distorsion(imgRGB,imgDepth)
                # cambiar de dominio rgb a depth
                imgRGB=self.rgb2_depth(imgRGB)
                imgGray=cv2.cvtColor(imgRGB,cv2.COLOR_BGR2GRAY)
                ret, mask= cv.threshold(imgGray,254,255,cv.THRESH_BINARY)
                z,uv=self.getPuntos(imgDepth,mask)
                
                zModelo=np.concatenate((z,zModelo))
                uvModelo=np.concatenate((uv,uvModelo))

                graficar([mask,imgDepth],1,2)
                imgDepth[mask!=255]=imgDepth[mask==255].mean()
                heatMap(imgDepth)
    
            #     heatMap(zModelo
        zModelo=self.calcularPlano(zModelo,uvModelo,self.shape)
        return np.array(zModelo,dtype=np.uint16)
    def save(self,img,path,name):
         cv2.imwrite("{}{}.png".format(path,name),img)
    def guardar(self,img,nombre,path=""):
        #crear objeto
        config = ConfigParser()
        #revisar existencia del archivo
        ret=config.read('{}{}.ini'.format(path,nombre))
    
        if ret: #verificar si el objeto esta vacio
            ret=True
        else:
            ret=False
       
        #toda la informacion se convierte a vector fila para facilitar luego su lectura.
      
       
        config['ref_img'] = {'media':f'[{img.mean()}]',
                             'shape':f'[{img.shape[0]} {img.shape[1]}]',
                            }
        pixeles=img.reshape(-1)
        for i,pixel in enumerate(pixeles):
            config.set('ref_img', "pixel_{}".format(i), str(pixel))
            
        #guardar archivo
        with open('{}{}.ini'.format(path,nombre), 'w') as configfile:
            config.write(configfile)
        return ret

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import cv2
import sys# las sisguientes dos lineas ayudan a ubicar al archivo calibracion.py
sys.path.append("/home/estufab4/Desktop/flujoBagazo/codigo/calibracion/")
from calibracion import graficar
from medicion import *

path_config="/home/estufab4/Desktop/flujoBagazo/codigo/calibracion/configuraciónPrueba/"
# "/home/estufab4/Desktop/flujo de bagazo/codigo/calibracion/configuracion_union_150/"
config="distorsionz"
getRef=GetRef(path_config,config)


In [ ]:
path_img="/home/estufab4/Desktop/flujoBagazo/dataset/img_dataset_bagazo/risaralda/septiembre/29/fondo2/"
zModelo=getRef.ref(path_img)

In [ ]:
heatMap(zModelo)
zModelo=cv2.merge([zModelo,zModelo,zModelo])
# getRef.save(zModelo,path_img,"fondo")
    


In [ ]:
getRef.guardar(zModelo,'fondo')

# Datos Risaralda

In [ ]:
tendencia1=tendencia[1:]
tendencia2=tendencia[1:]
tendencia3=tendencia[1:]

In [ ]:
import pandas as pd
from datetime import datetime
from scipy.signal import savgol_filter
ipath = "/home/estufab4/Desktop/flujo de bagazo/codigo/img_dataset/risaralda/Pruebas cenicaña agosto 2021.xlsx"
df = pd.read_excel(ipath,index_col=0,sheet_name=1)

lapso=["2021-08-04 12:00:00","2021-08-04 12:14:59"]
# lapso=["2021-08-04 11:40:00","2021-08-04 12:11:59"]

formato="%Y-%m-%d %H:%M:%S"
# df.index=pd.to_datetime(df.index)
# df.index = pd.to_datetime(index,format=formato)
    
def filtrarLapso(df,inicio,final):
    startDate = datetime.strptime(inicio,formato)
    endDate = datetime.strptime(final,formato)

    dfFiltro=df[(df.index>=startDate)&(df.index<=endDate)]
    return dfFiltro
df=filtrarLapso(df,lapso[0],lapso[1])
# print(df)
colum=df.columns
df[colum[0]]=savgol_filter(df[colum[0]], 11, 3, mode='nearest')
a=df.plot(y=colum[0],grid=True,figsize=(15,10))
plt.show()
group=df.resample("60S")[df.columns[0]]
fabrica=[]
for _,minute in group:
    minute=np.array(minute)
#     print(hora.sum())
#     plt.plot(hora)
#     plt.show()
    fabrica.append(minute.sum()/3600)
fabrica=np.array(fabrica)
print(fabrica.shape,tendencia.shape)


In [ ]:
def evaluar(tendencia,fabrica,d=0,h=0):
    if  h:
        d=4272-307*h+8.3*h**2-0.0977*h**3+(4.32/10000)*h**4
        print("densidad: ",d)
    tendencia=tendencia*d/1000
    print(len(fabrica),len(tendencia))
    plt.plot(fabrica*60, label="Fabrica")
    plt.plot(tendencia*60,label="Cámara")
    plt.grid()
    plt.title("Consumo")
    plt.xlabel("muestra (min)", size = 16,)
    plt.ylabel("Masa (T/H)", size = 16,)
    plt.legend()
    # grafica del error %

    error=100*(fabrica-tendencia)/fabrica
    print("consumo cáramara: ",tendencia.sum())
    print("consumo Fabrica: ",fabrica.sum())
    errorT=100*(fabrica.sum()-tendencia.sum())/fabrica.sum()
    print("error total: ",errorT )
    print("error medio",error.mean())
    correlacion=np.corrcoef(fabrica,tendencia)
    print("correlación",correlacion[1,0])
    plt.show()
    plt.plot(error)
    plt.title("Errror")
    plt.xlabel("muestra [min]", size = 16,)
    plt.ylabel(" [%]", size = 16,)
    plt.show()
    return tendencia,error

In [ ]:
def get_density(h):
    d=4272-307*h+8.3*h**2-0.0977*h**3+(4.32/10000)*h**4
    4.3200e-04 9.7700e-02 8.3000e+00 3.0700e+02 4.2712e+04
    return d
d=get_density(54)
print(d)


In [ ]:
# graficar dencidades
def estimarDyH(fabrica,tendencia):
    d=fabrica*1000/tendencia# masa/volumen
    ref=np.ones_like(d)
    plt.plot(d,label="Estimación")
    plt.plot(160*ref,label="160Kg/m³ (Hugot 1986:98)",color="k")
    plt.plot(153.3*ref,label="153.3 h49",color="c")
    plt.plot(165.8*ref,label="165.8 h51",color="r")
    
    plt.title("Densidad")
    plt.xlabel("muestra [min]", size = 16,)
    plt.ylabel(" [Kg/m³]", size = 16,)
    plt.grid()
    plt.legend()
    plt.show()
    
    h= 22.6+0.215*d-1.9E-4*d**2
    print(d.shape)
    plt.plot(h)
    plt.title("Humedad")
    plt.xlabel("muestra [min]", size = 16,)
    plt.ylabel(" [%]", size = 16,)
    plt.grid()
    plt.show()
    
# estimarDyH(fabrica,tendencia[1:])
estimarDyH(fabrica,tendenciaCo)

In [ ]:
tendencia1,error1=evaluar(tendencia[1:],fabrica,h=47)
tendencia2,error2=evaluar(tendencia[1:],fabrica,h=49)
tendencia3,error3=evaluar(tendencia[1:],fabrica,h=51)

In [ ]:
def comparar(listd,listLabel,titulo,ejes,c=0):
    if c:
        color=["g","c","r","k"]
    else:
        color=["b","g","c","r","k"]
        
    for d,label,c in zip(listd,listLabel,color):
        plt.plot(d,label=label,color=c)
    plt.title(titulo)
    plt.xlabel(ejes[0], size = 16,)
    plt.ylabel(ejes[1], size = 16,)
    plt.legend()
    plt.grid()
    plt.show()


In [ ]:
comparar([fabrica*60,tendencia1*60,tendencia2*60,tendencia3*60],["báscula","h48","h50","h52"],"Tendencias",["[min]","[T/h]"])
comparar([error1,error2,error3],["h48","h50","h52"],"Error",["[min]","[%]"])

In [ ]:
# agregar factor de corrección
def addFacCorre(tendencia):
    # porcentaje de datos mal recortados por porcentaje de bagazo perdido por peso medio los lotes
    malRecorte=(1-97.8388998/100)*0.5*tendencia.mean()
    malSegmentacionMedia=(1.408450704/100)*0.20*tendencia
    malSegmentacionBaja=(0.2012072435/100)*0.60*tendencia
    tendencia=tendencia+malRecorte+malSegmentacionBaja+malSegmentacionMedia
    tendencia=tendencia+0.09*tendencia
    return tendencia
tendenciaCo=addFacCorre(tendencia[1:])
    

In [ ]:
# tendencia1co,error1co=evaluar(tendenciaCo,fabrica,147.597312)
# tendencia2co,error2co=evaluar(tendenciaCo,fabrica,159.5)
# tendencia3co,error3co=evaluar(tendenciaCo,fabrica,172.416512)
tendencia1co,error1co=evaluar(tendenciaCo,fabrica,h=47)
tendencia2co,error2co=evaluar(tendenciaCo,fabrica,h=49)
tendencia3co,error3co=evaluar(tendenciaCo,fabrica,h=51)

In [ ]:
comparar([fabrica*60,tendencia1co*60,tendencia2co*60,tendencia3co*60],["báscula","h47","h49","h51"],"Tendencias",["[min]","[T/h]"])
comparar([error1co,error2co,error3co],["h47","h49","h51"],"Error",["[min]","[%]"],c=1)
print(error1co.mean(),error2co.mean(),error3co.mean())

In [ ]:
print("error medio: ", error2.mean())
print("error medio corregido: ", error2co.mean())
comparar([fabrica*60,tendencia2*60,tendencia2co*60],["báscula","h49","h49cor"],"Tendencias",["[min]","[T/h]"])
comparar([error2,error2co],["h49","h49cor"],"Error",["[min]","[%]"],c=1)
ref=

In [ ]:
errorMedio=100*(media-10.018562489788875*(261/(5*60)))/media # kg/lote (261 lotes/(5*60 s))
print(errorMedio)